# Claude Relay 综合测试

这个 Notebook 用于测试 Claude Relay 的核心功能，包括：
- 🔀 路由切换测试
- 🧠 代码调试能力
- ⚡ 代码生成能力
- 🛠️ 工具调用能力
- 📡 流式响应测试

## 环境准备

In [ ]:
import sys
import json
from pathlib import Path

# 添加工具包路径
sys.path.append('utils')

from test_client import ClaudeRelayTestClient
from route_manager import RouteManager

# 初始化测试客户端和路由管理器
client = ClaudeRelayTestClient()
route_manager = RouteManager()

print("🚀 测试环境初始化完成")
print(f"📡 服务健康状态: {'✅ 正常' if client.test_health() else '❌ 异常'}")

## 查看当前路由配置

In [ ]:
# 显示当前路由信息
route_manager.show_route_info()

---

# 🧠 代码调试能力测试

## 场景1: JavaScript 数组越界问题

测试模型能否识别并修复常见的编程错误。

In [ ]:
# JavaScript 数组越界调试
prompt_js_debug = """
我的 JavaScript 函数有问题，请帮我调试：

```javascript
function calculateTotal(items) {
  let total = 0;
  for (let i = 0; i <= items.length; i++) {
    total += items[i].price * items[i].quantity;
  }
  return total;
}

const cart = [
  { price: 10, quantity: 2 },
  { price: 5, quantity: 3 }
];

console.log(calculateTotal(cart));
```

这个函数有什么问题？请指出潜在的 bug 并提供修复建议。
"""

result_js = client.send_message(prompt_js_debug)

if result_js['success']:
    print("✅ 请求成功")
    print(f"⏱️ 响应时间: {result_js['response_time']:.2f}s")
    print("\n📝 AI 回复:")
    print(result_js['content'][:500] + "..." if len(result_js['content']) > 500 else result_js['content'])
    
    # 检查关键词
    keywords = ['数组越界', '<=', '修复', 'i < items.length']
    found_keywords = [kw for kw in keywords if kw.lower() in result_js['content'].lower()]
    print(f"\n🔍 关键词检测: {len(found_keywords)}/{len(keywords)} 个关键词匹配")
    print(f"   匹配: {found_keywords}")
else:
    print(f"❌ 请求失败: {result_js['error']}")

## 场景2: Python 异常处理问题

测试模型对 Python 代码潜在异常的识别能力。

In [ ]:
# Python 异常处理调试
prompt_py_debug = """
分析这个 Python 函数的潜在问题并提供改进建议：

```python
def process_user(data):
    name = data['name']
    age = int(data['age'])
    email = data['email']
    
    if age < 18:
        return f'{name} 未成年，无法注册'
    
    return f'{name} ({age}岁) 注册成功，邮箱: {email}'

# 使用示例
user_data = {'name': 'Alice', 'age': '25'}
result = process_user(user_data)
```

这个函数有什么潜在的问题？请考虑异常处理、数据验证等方面。
"""

result_py = client.send_message(prompt_py_debug)

if result_py['success']:
    print("✅ 请求成功")
    print(f"⏱️ 响应时间: {result_py['response_time']:.2f}s")
    print("\n📝 AI 回复:")
    print(result_py['content'][:500] + "..." if len(result_py['content']) > 500 else result_py['content'])
    
    # 检查关键词
    keywords = ['KeyError', '异常', '处理', 'try', 'except']
    found_keywords = [kw for kw in keywords if kw.lower() in result_py['content'].lower()]
    print(f"\n🔍 关键词检测: {len(found_keywords)}/{len(keywords)} 个关键词匹配")
    print(f"   匹配: {found_keywords}")
else:
    print(f"❌ 请求失败: {result_py['error']}")

---

# ⚡ 代码生成能力测试

## 场景3: React 组件生成

测试模型生成完整、可用的 React 组件的能力。

In [ ]:
# React 组件生成
prompt_react = """
请帮我创建一个 React 无限滚动组件，用于展示新闻列表。要求：

1. 使用 Intersection Observer API 实现无限滚动
2. 每次加载 20 条数据
3. 包含加载状态和错误处理
4. 完整的 TypeScript 类型支持
5. 使用现代 React Hooks

请提供完整的组件代码和使用示例。
"""

result_react = client.send_message(prompt_react)

if result_react['success']:
    print("✅ 请求成功")
    print(f"⏱️ 响应时间: {result_react['response_time']:.2f}s")
    print(f"📏 响应长度: {len(result_react['content'])} 字符")
    
    # 检查关键词
    keywords = ['IntersectionObserver', 'useState', 'useEffect', 'interface', 'TypeScript']
    found_keywords = [kw for kw in keywords if kw.lower() in result_react['content'].lower()]
    print(f"\n🔍 关键词检测: {len(found_keywords)}/{len(keywords)} 个关键词匹配")
    print(f"   匹配: {found_keywords}")
    
    # 显示代码片段预览
    print("\n📝 代码预览 (前 300 字符):")
    print(result_react['content'][:300] + "...")
else:
    print(f"❌ 请求失败: {result_react['error']}")

---

# 🛠️ 工具调用能力测试

## 场景4: 代码执行工具调用

测试模型使用工具执行代码的能力。

In [ ]:
# 工具调用测试
prompt_tool = """
请帮我计算斐波那契数列的第 15 项，使用代码执行工具来验证结果。
同时请解释斐波那契数列的计算原理和时间复杂度。
"""

tools = [
    {
        "name": "execute_python",
        "description": "执行 Python 代码并返回结果",
        "input_schema": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "要执行的 Python 代码"
                }
            },
            "required": ["code"]
        }
    }
]

result_tool = client.send_message(prompt_tool, tools=tools, tool_choice="auto")

if result_tool['success']:
    print("✅ 请求成功")
    print(f"⏱️ 响应时间: {result_tool['response_time']:.2f}s")
    
    # 检查是否包含工具调用
    raw_response = str(result_tool.get('raw_response', ''))
    has_tool_call = 'tool_use' in raw_response or 'execute_python' in raw_response
    
    print(f"🛠️ 工具调用检测: {'✅ 包含工具调用' if has_tool_call else '❌ 未检测到工具调用'}")
    
    # 检查关键词
    keywords = ['fibonacci', '610', 'execute_python']
    content_to_check = result_tool['content'] + raw_response
    found_keywords = [kw for kw in keywords if kw.lower() in content_to_check.lower()]
    print(f"🔍 关键词检测: {len(found_keywords)}/{len(keywords)} 个关键词匹配")
    print(f"   匹配: {found_keywords}")
    
    print("\n📝 响应内容:")
    print(result_tool['content'][:400] + "..." if len(result_tool['content']) > 400 else result_tool['content'])
else:
    print(f"❌ 请求失败: {result_tool['error']}")

---

# 📡 流式响应测试

## 场景5: 流式长文本生成

测试模型的流式响应能力。

In [ ]:
# 流式响应测试
prompt_stream = """
请详细解释微服务架构的设计原则和最佳实践，包括：

1. 服务拆分策略
2. 服务间通信机制
3. 数据一致性处理
4. 服务治理和监控
5. 容错和降级机制
6. 部署和运维策略

请结合具体的技术栈和实际案例进行说明，篇幅要详细全面。
"""

result_stream = client.send_message(prompt_stream, stream=True)

if result_stream['success']:
    print("✅ 请求成功")
    print(f"⏱️ 响应时间: {result_stream['response_time']:.2f}s")
    print(f"📡 响应类型: {result_stream['type']}")
    
    if result_stream['type'] == 'stream':
        print("✅ 检测到流式响应")
        print(f"📏 流式内容长度: {len(result_stream['content'])} 字符")
    else:
        print("⚠️ 未检测到流式响应，可能是普通响应")
        print(f"📏 响应长度: {len(result_stream['content'])} 字符")
    
    # 检查关键词
    keywords = ['微服务', '架构', '设计', '服务拆分', '通信']
    found_keywords = [kw for kw in keywords if kw.lower() in result_stream['content'].lower()]
    print(f"🔍 关键词检测: {len(found_keywords)}/{len(keywords)} 个关键词匹配")
    print(f"   匹配: {found_keywords}")
    
    print("\n📝 内容预览 (前 300 字符):")
    print(result_stream['content'][:300] + "...")
else:
    print(f"❌ 请求失败: {result_stream['error']}")

---

# 🔀 路由切换测试

## 测试不同路由配置下的表现差异

In [ ]:
# 简单的跨路由对比测试
test_prompt = "请用一句话介绍什么是人工智能。"

results_by_route = {}

# 测试不同路由
routes_to_test = ['gemini', 'openai']

for route_type in routes_to_test:
    print(f"\n🔀 切换到 {route_type.upper()} 路由")
    
    # 切换路由
    if route_manager.quick_switch(route_type):
        # 等待一下确保路由切换生效
        import time
        time.sleep(1)
        
        # 测试请求
        result = client.send_message(test_prompt)
        
        if result['success']:
            results_by_route[route_type] = {
                'response_time': result['response_time'],
                'content': result['content'],
                'length': len(result['content'])
            }
            print(f"✅ {route_type} 路由测试成功")
            print(f"   响应时间: {result['response_time']:.2f}s")
            print(f"   响应长度: {len(result['content'])} 字符")
            print(f"   内容预览: {result['content'][:100]}...")
        else:
            print(f"❌ {route_type} 路由测试失败: {result['error']}")
    else:
        print(f"❌ 切换到 {route_type} 路由失败")

# 对比结果
if len(results_by_route) > 1:
    print("\n📊 路由对比结果:")
    for route, data in results_by_route.items():
        print(f"   {route.upper()}: {data['response_time']:.2f}s, {data['length']} 字符")

---

# 📊 测试总结

## 生成完整的测试报告

In [ ]:
# 收集所有测试结果
all_results = []

# 添加之前的测试结果
test_cases = [
    ('JavaScript 调试', result_js if 'result_js' in locals() else None),
    ('Python 调试', result_py if 'result_py' in locals() else None),
    ('React 组件生成', result_react if 'result_react' in locals() else None),
    ('工具调用测试', result_tool if 'result_tool' in locals() else None),
    ('流式响应测试', result_stream if 'result_stream' in locals() else None)
]

print("📋 测试结果汇总:")
print("=" * 50)

total_tests = 0
successful_tests = 0
total_time = 0

for test_name, result in test_cases:
    if result is not None:
        total_tests += 1
        status = "✅ 成功" if result['success'] else "❌ 失败"
        response_time = result.get('response_time', 0)
        total_time += response_time
        
        if result['success']:
            successful_tests += 1
        
        print(f"{test_name:20} | {status} | {response_time:.2f}s")
    else:
        print(f"{test_name:20} | ⚪ 未执行")

print("=" * 50)

if total_tests > 0:
    success_rate = (successful_tests / total_tests) * 100
    avg_time = total_time / total_tests
    
    print(f"📊 总体统计:")
    print(f"   总测试数: {total_tests}")
    print(f"   成功数: {successful_tests}")
    print(f"   成功率: {success_rate:.1f}%")
    print(f"   平均响应时间: {avg_time:.2f}s")
else:
    print("📊 没有完成的测试")

# 路由测试结果
if 'results_by_route' in locals() and results_by_route:
    print(f"\n🔀 路由测试结果:")
    for route, data in results_by_route.items():
        print(f"   {route.upper()}: {data['response_time']:.2f}s")

## 🎯 结论

通过这个 Notebook，我们可以：

1. **直观地看到每个测试场景的 prompt** - 直接写在 Markdown 中
2. **实时执行测试** - 每个代码块都可以独立运行
3. **观察详细结果** - 包括响应时间、内容验证等
4. **灵活修改参数** - 可以随时调整 prompt 或测试参数
5. **切换路由配置** - 一键切换不同的路由进行对比
6. **生成测试报告** - 自动汇总所有测试结果

这种方式既保持了测试的严谨性，又提供了极佳的可读性和交互性！